# Similarity computations using w2v and d2v

Compute similarities among patents using word2vec and doc2vec models.  
Use the same datasets as `tfidf_nearest.ipynb`.

In [1]:
import h5py
import pandas as pd
import numpy as np
import pickle

In [2]:
citations_info_target = pd.read_pickle("../data/citations_info_2000.df.gz")
training_app_df = pd.read_pickle("../data/training_app_1000.df.gz")
testset_app_df = pd.read_pickle("../data/testset_app_1000.df.gz")
grants_target_df = pd.read_pickle("../data/grants_for_2000.df.gz")

In [3]:
import re
CLAIM_PAT = re.compile(r'<claims[^>]*>(.*)</claims>',re.MULTILINE|re.DOTALL)
TAG_PAT = re.compile(r"<.*?>")
def whole_xml_to_claim_xml(whole):
    mat = CLAIM_PAT.search(whole)
    return mat.group(1)
def whole_xml_to_claim(whole):
    return TAG_PAT.sub(' ', whole_xml_to_claim_xml(whole))

grants_target_df["claim"] = grants_target_df["xml"].map(whole_xml_to_claim)

In [4]:
def set_one_answer_appid(labeldf, oneappid):
    cited_patids = citations_info_target[citations_info_target.app_id == oneappid].parsed
    labeldf.loc[oneappid] = labeldf.columns.isin(cited_patids)
    
def create_label_df():
    label_df = pd.DataFrame(columns=grants_target_df.parsed.values, dtype=np.bool)
    for appid in testset_app_df.app_id:
        set_one_answer_appid(label_df, appid)
    return label_df

label_df = create_label_df()

In [5]:
def predict_test_set(predict_func):
    """
    predict_func(claims) return NxM of boolean. N is len(claims). M is rownum of grants_target_df.
            value indicate n claim is cite patent of m row of grants_all_df.
    """
    predictdf = pd.DataFrame(columns=grants_target_df.parsed.values, dtype=np.bool)
    res = predict_func(testset_app_df["xml"].map(whole_xml_to_claim))
    for idx, appid in enumerate(testset_app_df.app_id):
        predictdf.loc[appid] = res[idx, :]
    return predictdf

In [6]:
def calc_TPs(preddf, labeldf):
    return sum([sum(preddf.loc[one_appid][labeldf.loc[one_appid]]) for one_appid in testset_app_df.app_id])

def calc_FPs(preddf, labeldf):
    return sum([sum(preddf.loc[one_appid][~labeldf.loc[one_appid]]) for one_appid in testset_app_df.app_id])

def calc_TNs(preddf, labeldf):
    return sum([sum(preddf.loc[one_appid][~labeldf.loc[one_appid]] == False) for one_appid in testset_app_df.app_id])

def calc_FNs(preddf, labeldf):
    return sum([sum(preddf.loc[one_appid][labeldf.loc[one_appid]] == False) for one_appid in testset_app_df.app_id])

def calc_TFPNs(preddf, labeldf):
    return calc_TPs(preddf, labeldf), calc_FPs(preddf, labeldf), calc_TNs(preddf, labeldf), calc_FNs(preddf, labeldf)

In [7]:
def calc_summary_TFPNs(TP, FP, TN, FN):
    "return acc, prec, recall, f1."
    return pd.DataFrame(columns=["acc", "prec", "recall", "f1"], data=[[(TP+TN)/(TP+FP+TN+FN), TP/(TP+FP), TP/(TP+FN), 2*TP/(2*TP+FP+FN)]])
    
def calc_summary(preddf, labeldf):
    TP, FP, TN, FN = calc_TFPNs(preddf, labeldf)
    return calc_summary_TFPNs(TP, FP, TN, FN)

# Word2Vec model

In [8]:
from gensim.models import Word2Vec
from scipy import spatial

import multiprocessing
CPUNUM = multiprocessing.cpu_count()

In [9]:
grants_target_claims = grants_target_df['claim'].map(lambda x:x.split()).tolist()

In [10]:
%%time

w2v = Word2Vec(grants_target_claims, size=100, window=5, min_count=5, workers=CPUNUM, iter=10, hs=1)

CPU times: user 2min 24s, sys: 425 ms, total: 2min 25s
Wall time: 39.2 s


In [11]:
w2v.most_similar('machine')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('machines', 0.498989462852478),
 ('machine.', 0.4812130630016327),
 ('machine,', 0.4266325831413269),
 ('sub-system', 0.42260175943374634),
 ('engine', 0.4178369343280792),
 ('sub-system,', 0.416022390127182),
 ('system', 0.4014444351196289),
 ('arrangement', 0.3917171359062195),
 ('medium', 0.3898860216140747),
 ('registry', 0.38966041803359985)]

In [12]:
grants_ids = columns=grants_target_df.parsed.values

In [13]:
np.average(w2v[['machine', 'water']], axis=0)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.02874494,  1.5051155 ,  2.8695116 ,  0.6352596 ,  1.8710206 ,
        0.22065717, -0.7221451 , -1.795708  ,  0.6495555 , -2.4453335 ,
        0.5088709 ,  1.5201758 ,  1.7536943 , -0.21682355,  0.16243666,
        2.3606777 ,  1.1397797 , -0.37961718, -1.9169936 ,  0.16223368,
        0.66769606, -0.714554  , -1.0434326 , -1.985987  ,  2.4417868 ,
       -0.23684612,  1.1743438 , -0.3844646 ,  1.3798729 ,  0.13665694,
       -0.7199033 , -0.61454964, -0.25440404,  1.2129455 ,  1.1049573 ,
       -0.14381033, -0.49564204,  2.190461  , -1.1458721 ,  0.36150506,
        0.24387074, -0.31758544, -0.43413782,  0.90190035, -0.14067854,
        0.5365779 ,  0.77395993, -0.5550861 , -0.11838039,  1.2492237 ,
        2.2935867 , -0.2386921 ,  0.02345926,  1.2740526 , -1.1083058 ,
        0.02743721,  0.32710654,  0.1868825 , -1.0381557 ,  1.6341794 ,
       -0.26616442,  1.4961631 ,  0.45894694, -0.9184972 ,  0.24134654,
        0.7245893 , -0.69919866, -0.92366385, -0.06156743,  0.24

In [14]:
def text_to_vec(text):
    words = text.split()
    filtered_words = []
    for word in words:
        if word in w2v.wv.vocab:
            filtered_words.append(word)
    vec = np.average(w2v[filtered_words], axis=0)
    return vec

In [15]:
grants_w2v_vectors = []
for i, id in enumerate(grants_ids):
    vec = text_to_vec(grants_target_df.loc[i]['claim'])
    grants_w2v_vectors.append(vec)
grants_w2v_vectors = np.array(grants_w2v_vectors)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [16]:
np.argsort([6,1,2,4,5])[::-1][0:2]

array([0, 4])

In [17]:
def find_most_similar(vec):
    topN = 5
    similarities = []
    for grants_vec in grants_w2v_vectors:
        sim = 1 - spatial.distance.cosine(vec, grants_vec)
        similarities.append(sim)
    return [np.argmax(similarities)]
#     return np.argsort(similarities)[::-1][0:topN]

In [18]:
find_most_similar(grants_w2v_vectors[2,:])

[2]

In [19]:
def predict_test_set():
    predictdf = pd.DataFrame(columns=grants_target_df.parsed.values, dtype=np.bool)
    for idx, appid in enumerate(testset_app_df.app_id):
        text = whole_xml_to_claim(testset_app_df.loc[idx]['xml'])
        vec = text_to_vec(text)
        similar_grant_indexes = find_most_similar(vec)
        predictdf.loc[appid] = False
        for similar_grant_index in similar_grant_indexes:
            similar_grant_id = grants_ids[similar_grant_index]
            predictdf.loc[appid, similar_grant_id] = True
        if idx%100 == 0:
            print(idx)
    return predictdf

In [20]:
%%time

pred_df = predict_test_set()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


0
100
200
300
400
500
600
700
800
900
CPU times: user 2min 47s, sys: 7.31 s, total: 2min 54s
Wall time: 2min 54s


In [21]:
pred_df.head()

,6837383,6837647,6837799,6837893,6837910,6838140,6838207,6838507,6838812,6838925,...,8334161,8334431,8334887,8336128,8336158,8336789,8336964,8337193,8339697,8340894
14307191,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
13137006,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12741959,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
12643447,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
14200253,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [22]:
# top1
calc_summary(pred_df, label_df)

,acc,prec,recall,f1
0,0.999216,0.136,0.108713,0.120835


In [114]:
# # top5
# calc_summary(pred_df, label_df)

,acc,prec,recall,f1
0,0.997756,0.0588,0.235012,0.094065


## Example

In [23]:
testset_app_df.head()

,app_id,xml
0,14307191,"<us-patent-application lang=""EN"" dtd-version=""..."
1,13137006,"<us-patent-application lang=""EN"" dtd-version=""..."
2,12741959,"<us-patent-application lang=""EN"" dtd-version=""..."
3,12643447,"<us-patent-application lang=""EN"" dtd-version=""..."
4,14200253,"<us-patent-application lang=""EN"" dtd-version=""..."


In [24]:
idx = 0
appid = 14307191
text = whole_xml_to_claim(testset_app_df.loc[idx]['xml'])
vec = text_to_vec(text)
similar_grant_index = find_most_similar(vec)
similar_grant_id = grants_ids[similar_grant_index]
print(text)
print(grants_target_df.loc[similar_grant_index]['claim'])


 
  1 . A method to aggregate, filter, and share energy data for analysis, the method comprising:
 receiving first data associated with a first electrical circuit, the first data having a first protocol; 
 sampling the first data at a first sampling rate to generate first digital data, wherein the first sampling rate is substantially continuous; and 
 transmitting reporting digital data over a network having a network protocol different from the first protocol, the reporting digital data comprising at least the first digital data, wherein the reporting digital data is transmitted at a reporting rate that is decoupled from the first sampling rate. 
 
 
 
  2 . The method of  claim 1  further comprising:
 receiving second data associated with a second electrical circuit, the second data having a second protocol different from the first protocol; and 
 sampling the second data at a second sampling rate to generate second digital data, wherein the second sampling rate is substantially con

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [25]:
idx = 1
text = whole_xml_to_claim(testset_app_df.loc[idx]['xml'])
vec = text_to_vec(text)
similar_grant_index = find_most_similar(vec)
similar_grant_id = grants_ids[similar_grant_index]
print(text)
print(grants_target_df.loc[similar_grant_index]['claim'])


 
  1 . A display apparatus, comprising:
 a position sensor to sense an eye position of a user; 
 a controller to set a virtual viewing window corresponding to the sensed eye position of the user and to provide a control signal to generate a directional light toward the virtual viewing window; and 
 a light generator to generate a directional light based on the control signal. 
 
 
 
  2 . The display apparatus of  claim 1 , further comprising:
 a light modulator to modulate an intensity of the directional light based on the control signal. 
 
 
 
  3 . The display apparatus of  claim 1 , wherein the position sensor comprises at least one camera to photograph the eye position of the user. 
 
 
  4 . The display apparatus of  claim 1 , wherein the position sensor comprises at least one camera to identify the eye position of the user by photographing an identifier fixed around the eye position of the user. 
 
 
  5 . The display apparatus of  claim 1 , wherein the light generator compri

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


## Trial and error

In [26]:
predictdf = pd.DataFrame(columns=grants_target_df.parsed.values, dtype=np.bool)

In [27]:
predictdf.loc[14307191] = False

In [28]:
predictdf.loc[14307191, 6837383] = True

In [29]:
predictdf

,6837383,6837647,6837799,6837893,6837910,6838140,6838207,6838507,6838812,6838925,...,8334161,8334431,8334887,8336128,8336158,8336789,8336964,8337193,8339697,8340894
14307191,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [30]:
predictdf.loc[14307191, 6837383]

True

# Doc2vec model

In [26]:
grants_target_df['claim'][0]

'\n \n 1. A tool organizer for mounting to and adapted for use with a vehicle lift rack having support arms, said tool organizer comprising:\n four upstanding walls depending from a base and forming a storage volume for storing a plurality of tools and parts;  \n a selectively articulating lid enclosing said storage volume;  \n adjustable attachment means depending from said storage volume, said attachment means removably attachable to said support arms for supporting said tool organizer by gravity impingement;  \n a U-shaped ring affixed to an external portion of at least one of said walls, said ring for supporting a hand tool;  \n a support plate projected from a front of said tool organizer and co-extensive with said base;  \n a plurality of intermediate apertures formed in said support plate, said plurality of intermediate apertures adapted sized to accommodate intermediate and large hand tools;  \n a plurality of small apertures formed in said support plate, said plurality of smal

### Do simple text preprocessing and create datasets for doc2vec model.

In [27]:
preprocessed_grant_data = grants_target_df['claim'].str\
    .replace("\n", "", regex=False).str\
    .replace("[0-9]*\.", "", regex=True).str\
    .replace("[,:;]", "", regex=True).str\
    .replace("a |A |the |The ", "", regex=True)

In [28]:
preprocessed_grant_data[0]

'   tool organizer for mounting to and adapted for use with vehicle lift rack having support arms said tool organizer comprising four upstanding walls depending from base and forming storage volume for storing plurality of tools and parts   selectively articulating lid enclosing said storage volume   adjustable attachment means depending from said storage volume said attachment means removably attachable to said support arms for supporting said tool organizer by gravity impingement   U-shaped ring affixed to an external portion of at least one of said walls said ring for supporting hand tool   support plate projected from front of said tool organizer and co-extensive with said base   plurality of intermediate apertures formed in said support plate said plurality of intermediate apertures adapted sized to accommodate intermediate and large hand tools   plurality of small apertures formed in said support plate said plurality of small apertures adapted to accommodate small hand tools   at

In [29]:
from gensim.models import doc2vec
from gensim.utils import simple_preprocess

In [30]:
d2v_grant_data = list()

for idx, elem in enumerate(preprocessed_grant_data):
    d2v_grant_data.append(
        doc2vec.TaggedDocument(simple_preprocess(elem), [str(grants_target_df.parsed[idx])])
    )

In [31]:
d2v_grant_data[0]

TaggedDocument(words=['tool', 'organizer', 'for', 'mounting', 'to', 'and', 'adapted', 'for', 'use', 'with', 'vehicle', 'lift', 'rack', 'having', 'support', 'arms', 'said', 'tool', 'organizer', 'comprising', 'four', 'upstanding', 'walls', 'depending', 'from', 'base', 'and', 'forming', 'storage', 'volume', 'for', 'storing', 'plurality', 'of', 'tools', 'and', 'parts', 'selectively', 'articulating', 'lid', 'enclosing', 'said', 'storage', 'volume', 'adjustable', 'attachment', 'means', 'depending', 'from', 'said', 'storage', 'volume', 'said', 'attachment', 'means', 'removably', 'attachable', 'to', 'said', 'support', 'arms', 'for', 'supporting', 'said', 'tool', 'organizer', 'by', 'gravity', 'impingement', 'shaped', 'ring', 'affixed', 'to', 'an', 'external', 'portion', 'of', 'at', 'least', 'one', 'of', 'said', 'walls', 'said', 'ring', 'for', 'supporting', 'hand', 'tool', 'support', 'plate', 'projected', 'from', 'front', 'of', 'said', 'tool', 'organizer', 'and', 'co', 'extensive', 'with', 'said

### doc2vec model training.

model parameters: https://radimrehurek.com/gensim/models/doc2vec.html

In [32]:
import multiprocessing
CPUNUM = multiprocessing.cpu_count()

In [33]:
%%time

model = doc2vec.Doc2Vec(
    documents=d2v_grant_data
    , dm = 1
    , epochs = 50
    , vector_size=350
    , window=5
    , min_count=5
    , workers=CPUNUM
    , seed=23)

CPU times: user 9min 1s, sys: 4.48 s, total: 9min 6s
Wall time: 2min 28s


Check word similarities.

In [34]:
model.most_similar("machine")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('computer', 0.3113054037094116),
 ('mediaccording', 0.22077366709709167),
 ('medias', 0.20563620328903198),
 ('writable', 0.2033187448978424),
 ('markup', 0.20132245123386383),
 ('abradable', 0.20126992464065552),
 ('wired', 0.19824786484241486),
 ('demanding', 0.1957094520330429),
 ('hf', 0.1904827207326889),
 ('processor', 0.18401512503623962)]

It looks nice.  
Next check document similarities.

In [35]:
grants_target_df.head(2)

,parsed,xml,claim
0,6837383,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",\n \n 1. A tool organizer for mounting to and ...
1,6837647,"<us-patent-grant lang=""EN"" dtd-version=""v40 20...",\n \n 1. A modular crowd and traffic control b...


In [36]:
idx = 6837383

In [37]:
model.docvecs.most_similar(str(idx))

[('6860386', 0.43970587849617004),
 ('7258401', 0.43385937809944153),
 ('7383954', 0.42699262499809265),
 ('7242108', 0.39295336604118347),
 ('7213816', 0.3670129179954529),
 ('8192098', 0.3617875874042511),
 ('7543681', 0.3472362160682678),
 ('7007804', 0.3378317058086395),
 ('7181887', 0.3360327184200287),
 ('6971511', 0.33588314056396484)]

In [38]:
most_similar_grant_idx = int(model.docvecs.most_similar(str(idx))[0][0])

In [39]:
most_similar_grant_idx

6860386

Check similar documents.

In [40]:
grants_target_df[grants_target_df.parsed == idx]['claim'].values

array(['\n \n 1. A tool organizer for mounting to and adapted for use with a vehicle lift rack having support arms, said tool organizer comprising:\n four upstanding walls depending from a base and forming a storage volume for storing a plurality of tools and parts;  \n a selectively articulating lid enclosing said storage volume;  \n adjustable attachment means depending from said storage volume, said attachment means removably attachable to said support arms for supporting said tool organizer by gravity impingement;  \n a U-shaped ring affixed to an external portion of at least one of said walls, said ring for supporting a hand tool;  \n a support plate projected from a front of said tool organizer and co-extensive with said base;  \n a plurality of intermediate apertures formed in said support plate, said plurality of intermediate apertures adapted sized to accommodate intermediate and large hand tools;  \n a plurality of small apertures formed in said support plate, said plurality 

In [41]:
grants_target_df[grants_target_df.parsed == most_similar_grant_idx]['claim'].values

array(['\n \n 1. A jewelry box for containing jewelry comprising the combination of:\n a multi-sided container defining an interior chamber and having a frontal opening of lesser dimension than the width of said container;  \n said interior chamber having retaining means to slidably receive a slidable drawer;  \n a slidable drawer received within said interior chamber in operative association with said retaining means;  \n said slidable drawer being of about the width of said frontal opening; and  \n at least one container adjacent said frontal opening and being movable to a first hidden position wholly within said interior chamber and a second accessible position once said slidable drawer is removed outwardly from said interior chamber, further wherein  \n said slidable drawer is removable from said interior chamber and wherein said container is pivotally mounted adjacent said frontal opening and is movable from a first hidden position wholly within said interior chamber into a second

Are these similar? It's not easy to judge, but these are both physical devices?

Let's continue to check model predictions

### Model prediction for test app data

Create the same type data of testset_app_df as we did for grant_target_df.

In [42]:
testset_app_df.head()

,app_id,xml
0,14307191,"<us-patent-application lang=""EN"" dtd-version=""..."
1,13137006,"<us-patent-application lang=""EN"" dtd-version=""..."
2,12741959,"<us-patent-application lang=""EN"" dtd-version=""..."
3,12643447,"<us-patent-application lang=""EN"" dtd-version=""..."
4,14200253,"<us-patent-application lang=""EN"" dtd-version=""..."


In [43]:
testset_app_df['claim'] = testset_app_df["xml"].map(whole_xml_to_claim)

In [44]:
testset_app_df.head()

,app_id,xml,claim
0,14307191,"<us-patent-application lang=""EN"" dtd-version=""...","\n \n 1 . A method to aggregate, filter, and ..."
1,13137006,"<us-patent-application lang=""EN"" dtd-version=""...","\n \n 1 . A display apparatus, comprising:\n ..."
2,12741959,"<us-patent-application lang=""EN"" dtd-version=""...",\n \n 1 - 33 . (canceled) \n \n \n 34 . A co...
3,12643447,"<us-patent-application lang=""EN"" dtd-version=""...",\n \n 1 . A terminal fitting formed by bendin...
4,14200253,"<us-patent-application lang=""EN"" dtd-version=""...",\n \n 1 . A printer for printing a three-dime...


In [45]:
preprocessed_testset_app_data = testset_app_df['claim'].str\
    .replace("\n", "", regex=False).str\
    .replace("[0-9]*\.", "", regex=True).str\
    .replace("[,:;]", "", regex=True).str\
    .replace("a |A |the |The ", "", regex=True)

In [46]:
preprocessed_testset_app_data = testset_app_df['claim']

In [47]:
vec = model.infer_vector( simple_preprocess(preprocessed_testset_app_data[0]) )

In [48]:
vec

array([ 0.32864848,  2.3249834 , -1.4818093 ,  1.1660504 ,  2.3678455 ,
       -1.2087846 , -4.7118244 , -0.76799035,  3.300299  ,  1.5128157 ,
        2.3943784 ,  6.304047  ,  0.68984646, -3.712247  , -4.3760204 ,
        1.4499943 , -1.7258772 ,  4.1700077 ,  1.0374267 , -0.8251143 ,
       -3.1510189 ,  0.2919668 , -1.7713672 , -0.7793044 ,  4.401563  ,
        0.38521072, -1.3831203 ,  4.178462  ,  1.7535076 , -3.8633177 ,
       -2.1812136 ,  0.7746414 ,  1.0867995 ,  4.5147786 , -0.80532897,
        0.49474466, -0.02519154, -1.090921  , -1.7264874 ,  1.51924   ,
       -6.0807357 ,  1.967718  , -3.9913354 , -1.4897691 ,  0.01724067,
       -2.8635764 , -4.089544  ,  1.0667527 , -0.4291979 ,  2.8537328 ,
       -0.1969331 ,  1.828931  ,  0.9831131 , -5.389817  , -1.2886466 ,
        1.2674333 , -3.3954635 , -1.6687337 ,  0.7988473 ,  2.178783  ,
        2.49024   , -1.3561416 , -0.2949141 , -3.7076628 ,  0.9848765 ,
        2.6711106 ,  2.5804334 , -2.6322978 ,  0.48035857, -1.20

In [49]:
model.docvecs.most_similar([vec])

[('8214829', 0.27610987424850464),
 ('7453937', 0.26095449924468994),
 ('7825815', 0.25970780849456787),
 ('8104041', 0.2513359785079956),
 ('6914242', 0.23791734874248505),
 ('7290193', 0.2362617701292038),
 ('7252786', 0.23244567215442657),
 ('7495935', 0.2295183390378952),
 ('8119152', 0.2290225625038147),
 ('7057392', 0.2289874106645584)]

In [50]:
[int(grantid) for grantid,similarity in model.docvecs.most_similar([vec], topn=1)]

[8214829]

In [51]:
def predict_test_set():
    predictdf = pd.DataFrame(columns=grants_target_df.parsed.values, dtype=np.bool)
    for idx, appid in enumerate(testset_app_df.app_id):
        text = preprocessed_testset_app_data[idx]
        vec = model.infer_vector(simple_preprocess(text))
        
        similar_grant_indexes = [int(grantid) 
                                 for grantid, similarity 
                                 in model.docvecs.most_similar([vec], topn=1)]

        predictdf.loc[appid] = False
        for similar_grant_index in similar_grant_indexes:
            predictdf.loc[appid, similar_grant_index] = True
        if idx%100 == 0:
            print(idx)
    
    return predictdf

In [52]:
%%time

pred_df = predict_test_set()

0
100
200
300
400
500
600
700
800
900
CPU times: user 4min 7s, sys: 4min 14s, total: 8min 22s
Wall time: 2min 39s


In [53]:
calc_summary(pred_df, label_df)

,acc,prec,recall,f1
0,0.999246,0.174,0.139089,0.154598
